<a href="https://colab.research.google.com/github/webrockerz2020/smart_health_assistant/blob/main/8_Predicting_and_Primitive_care.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fuzzywuzzy

In [3]:
import pickle
import pandas as pd
from fuzzywuzzy import process

import os
#Connecting the driver
from google.colab import drive

drive.mount('/content/drive/')

root_dir = '/content/drive/MyDrive/DAP/python_scripts'

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Mounted at /content/drive/


In [4]:
# getting primitive care
df_primitive_care = pd.read_csv(os.path.join(root_dir,"prec_t.csv"))
diagnose_list = df_primitive_care['diagnose'].tolist()
df_primitive_care.head(3)

,did,diagnose,pid
0,1,Abdominal aortic aneurysm: (enlarged major blo...,Stopping smoking may slow their growth. Small ...
1,2,Abdominal swelling,Eating smaller meals may help prevent bloating...
2,3,Abdominal trauma,Surgery.


In [5]:
def predicting_disease(syp,diagnose_list,feature_names):
    columns_list=feature_names.copy()
    for col in syp:
        if col not in feature_names:
            #print(col)
            syp.remove(col)

    temp_df = pd.DataFrame(0, index=[0], columns=columns_list)
    temp_df.loc[0, syp] = 1
    print(syp)
    print(temp_df)
    predicted_class_1 = loaded_model.predict(temp_df)
    #print("Predicted class:", predicted_class_1)
    # getting primitive care
    predicted_class_1 = str(predicted_class_1)
    diagnose_list = [str(item) for item in diagnose_list]
    best_match, similarity_score = process.extractOne(predicted_class_1, diagnose_list)
    threshold = 70
    if similarity_score >= threshold:
        print(f"Best match for '{predicted_class_1}': '{best_match}' with similarity score: {similarity_score}")
        primitive_care= df_primitive_care[df_primitive_care['diagnose']==best_match]
        primitive_care = primitive_care['pid'].iloc[0]
        message = f"Based on the symptoms you entered, there are chances that you are suffering from **{predicted_class_1}**.\n"\
          f"\nThe primitive care we would suggest is **{primitive_care}** \nThis recommendation is based on the symptoms you provided.\n "\
        "However, please note that this is not a substitute for professional medical advice. We strongly recommend consulting a healthcare professional "\
          "for further clarification and a proper diagnosis. Your health is important, and a medical professional can provide personalized guidance and care."
        print(message)

    else:
        print(f" Diseases Predicted '{predicted_class_1}' in the is not monitored.")


# Testing using Random Forest

## Logistic Regression

In [6]:
with open(os.path.join(root_dir,'logistic_regression_model_rd.pkl'), 'rb') as file:
    loaded_model = pickle.load(file)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.3.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
with open(os.path.join(root_dir,'rand_for_features_lr.txt')) as file:
    lines = [line.rstrip() for line in file]

In [8]:
feature_names_rf = lines.copy()

In [9]:
syp = ['facial paresis','muscle twitch','chill','fever','cough','decreased body weight']

In [10]:
predicting_disease(syp,diagnose_list,feature_names_rf)

['facial paresis', 'muscle twitch', 'chill', 'fever', 'cough', 'decreased body weight']
   pain  patient non compliance  fever  nausea  rale  pain abdominal  \
0     0                       0      1       0     0               0   

   shortness of breath  pain chest  night sweat  hypotension  ...  \
0                    0           0            0            0  ...   

   hypersomnia  focal seizures  dysuria  hacking cough  tremor  heartburn  \
0            0               0        0              0       0          0   

   painful swallowing  sinus rhythm  asthenia  dyspnea on exertion  
0                   0             0         0                    0  

[1 rows x 100 columns]
Best match for '["Alzheimer's disease"]': 'Alzheimer's disease: dementia' with similarity score: 90
Based on the symptoms you entered, there are chances that you are suffering from **["Alzheimer's disease"]**.

The primitive care we would suggest is **Rehabilitation and Occupational Therapy, Cognition-enhancin

In [11]:
syp = ['abscess bacterial','muscle twitch','chill','fever','barking cough']
predicting_disease(syp,diagnose_list,feature_names_rf)
# in this case "abscess bacterial" is not the correct syp

['muscle twitch', 'chill', 'fever']
   pain  patient non compliance  fever  nausea  rale  pain abdominal  \
0     0                       0      1       0     0               0   

   shortness of breath  pain chest  night sweat  hypotension  ...  \
0                    0           0            0            0  ...   

   hypersomnia  focal seizures  dysuria  hacking cough  tremor  heartburn  \
0            0               0        0              0       0          0   

   painful swallowing  sinus rhythm  asthenia  dyspnea on exertion  
0                   0             0         0                    0  

[1 rows x 100 columns]
Best match for '['cellulitis']': 'Cellulitis: skin infection' with similarity score: 90
Based on the symptoms you entered, there are chances that you are suffering from **['cellulitis']**.

The primitive care we would suggest is **Antibiotics and Penicillin** 
This recommendation is based on the symptoms you provided.
 However, please note that this is not a su

# Testing using Low Variance

## Logistic Regression

In [12]:
with open(os.path.join(root_dir,'low_var_features_lr.txt')) as file:
    lines = [line.rstrip() for line in file]

In [13]:
feature_names_lv= lines.copy()

In [14]:
syp = ['facial paresis','muscle twitch','chill','fever','cough','decreased body weight']
predicting_disease(syp,diagnose_list,feature_names_rf)

['facial paresis', 'muscle twitch', 'chill', 'fever', 'cough', 'decreased body weight']
   pain  patient non compliance  fever  nausea  rale  pain abdominal  \
0     0                       0      1       0     0               0   

   shortness of breath  pain chest  night sweat  hypotension  ...  \
0                    0           0            0            0  ...   

   hypersomnia  focal seizures  dysuria  hacking cough  tremor  heartburn  \
0            0               0        0              0       0          0   

   painful swallowing  sinus rhythm  asthenia  dyspnea on exertion  
0                   0             0         0                    0  

[1 rows x 100 columns]
Best match for '["Alzheimer's disease"]': 'Alzheimer's disease: dementia' with similarity score: 90
Based on the symptoms you entered, there are chances that you are suffering from **["Alzheimer's disease"]**.

The primitive care we would suggest is **Rehabilitation and Occupational Therapy, Cognition-enhancin

In [15]:
syp = ['abscess bacterial','muscle twitch','chill','fever','barking cough']
predicting_disease(syp,diagnose_list,feature_names_rf)

['muscle twitch', 'chill', 'fever']
   pain  patient non compliance  fever  nausea  rale  pain abdominal  \
0     0                       0      1       0     0               0   

   shortness of breath  pain chest  night sweat  hypotension  ...  \
0                    0           0            0            0  ...   

   hypersomnia  focal seizures  dysuria  hacking cough  tremor  heartburn  \
0            0               0        0              0       0          0   

   painful swallowing  sinus rhythm  asthenia  dyspnea on exertion  
0                   0             0         0                    0  

[1 rows x 100 columns]
Best match for '['cellulitis']': 'Cellulitis: skin infection' with similarity score: 90
Based on the symptoms you entered, there are chances that you are suffering from **['cellulitis']**.

The primitive care we would suggest is **Antibiotics and Penicillin** 
This recommendation is based on the symptoms you provided.
 However, please note that this is not a su